<a href="https://colab.research.google.com/github/o-rma/ML-FLASK-Pythonanywhere/blob/master/%E9%87%8D%E5%9B%9E%E5%B8%B0%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get -y install fonts-ipafont-gothic
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-ipafont-mincho
The following NEW packages will be installed:
  fonts-ipafont-gothic fonts-ipafont-mincho
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 8,237 kB of archives.
After this operation, 28.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-ipafont-gothic all 00303-21ubuntu1 [3,513 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-ipafont-mincho all 00303-21ubuntu1 [4,724 kB]
Fetched 8,237 kB in 1s (14.1 MB/s)
Selecting previously unselected package fonts-ipafont-gothic.
(Reading database ... 126102 files and directories currently installed.)
Preparing to unpack .../fonts-ipafont-gothic_00303-21ubuntu1_all.deb ...
Unpacking fonts-ipafont-gothic (00303-21ubuntu1) ...
Selecting previously unselected package fonts-ipaf

In [2]:


import matplotlib.font_manager as fm

# すべての TTF フォントの中から日本語フォントっぽい名前を表示
font_paths = fm.findSystemFonts(fontpaths=None, fontext='ttf')

print("=== 利用可能な日本語フォント名 ===")
for path in font_paths:
    try:
        font = fm.FontProperties(fname=path)
        name = font.get_name()
        if "IPA" in name or "Gothic" in name or "Noto" in name or "Hiragino" in name or "Yu" in name:
            print(name)
    except Exception:
        continue


=== 利用可能な日本語フォント名 ===
IPAMincho
IPAGothic
IPAGothic
IPAMincho
IPAPMincho
IPAPGothic


In [3]:
import matplotlib.font_manager as fm

# 使用したいフォント名に合わせて filter
target_font_name = "IPAGothic"  # ご自身の環境で見つかった名前に変更
font_path = None

for font in fm.findSystemFonts(fontpaths=None, fontext='ttf'):
    try:
        if fm.FontProperties(fname=font).get_name() == target_font_name:
            font_path = font
            break
    except Exception:
        continue

print("使用するフォントパス:", font_path)


使用するフォントパス: /usr/share/fonts/truetype/fonts-japanese-gothic.ttf


In [4]:
import pandas as pd
import glob
from sklearn.linear_model import LinearRegression
from google.colab import drive, files
import shutil
import os

# 1. Google Drive をマウント
drive.mount('/content/drive')

# データフォルダ内のすべてのExcelファイルを読み込む
file_paths = glob.glob("/content/drive/MyDrive/気象データと熱中症件数まとめ/*.xlsx")  # パスはあなたの環境に合わせて変更

data_list = []

for path in file_paths:
    try:
        df = pd.read_excel(path, sheet_name="Sheet1")  # シート名が統一されている前提
        df["都道府県"] = path.split("/")[-1].replace(".xlsx", "")  # ファイル名から都道府県を取得
        data_list.append(df)
    except Exception as e:
        print(f"{path} の読み込みでエラー: {e}")

# 全ファイル分のデータを結合
all_data = pd.concat(data_list, ignore_index=True)

# 数値変換（エラーは NaN に）
all_data["平均気温"] = pd.to_numeric(all_data["平均気温"], errors="coerce")
all_data["平均湿度"] = pd.to_numeric(all_data["平均湿度"], errors="coerce")
all_data["件数"] = pd.to_numeric(all_data["件数"], errors="coerce")

# 欠損値の除去
df_clean = all_data.dropna(subset=["平均気温", "平均湿度", "件数"])

# 重回帰分析（全国データで）
X = df_clean[["平均気温", "平均湿度"]]
y = df_clean["件数"]

model = LinearRegression()
model.fit(X, y)

# 結果の出力
coef = model.coef_
intercept = model.intercept_

print("=== 全国47都道府県のデータを使った重回帰分析結果 ===")
print(f"回帰係数: 平均気温 = {coef[0]:.2f}, 平均湿度 = {coef[1]:.2f}")
print(f"切片: {intercept:.2f}")
print(f"重回帰式: 熱中症件数 = {coef[0]:.2f} * 平均気温 + {coef[1]:.2f} * 平均湿度 + {intercept:.2f}")
r2 = model.score(X, y)
print(f"決定係数 R²: {r2:.4f}")

Mounted at /content/drive
=== 全国47都道府県のデータを使った重回帰分析結果 ===
回帰係数: 平均気温 = 2.65, 平均湿度 = -0.11
切片: -47.70
重回帰式: 熱中症件数 = 2.65 * 平均気温 + -0.11 * 平均湿度 + -47.70
決定係数 R²: 0.2068


In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# データの読み込み（既に df_clean に前処理済みと仮定）
# 例：df_clean = pd.read_excel(...)

# 説明変数と目的変数
X = df_clean[["平均気温", "平均湿度"]]
y = df_clean["件数"]

# モデルの学習
model = LinearRegression()
model.fit(X, y)

# 予測値
y_pred = model.predict(X)

# 回帰係数と切片
print("回帰係数:", model.coef_)
print("切片:", model.intercept_)
print(f"重回帰式: 件数 = {model.coef_[0]:.2f} * 平均気温 + {model.coef_[1]:.2f} * 平均湿度 + {model.intercept_:.2f}")

# 決定係数（R²）
r2 = model.score(X, y)
print(f"決定係数 R²: {r2:.4f}")

# 相関係数
corr = np.corrcoef(y, y_pred)[0, 1]
print(f"相関係数: {corr:.4f}")


回帰係数: [ 2.6508536  -0.11317862]
切片: -47.69503752096871
重回帰式: 件数 = 2.65 * 平均気温 + -0.11 * 平均湿度 + -47.70
決定係数 R²: 0.2068
相関係数: 0.4548


In [8]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression

# フォルダパスを指定（必要に応じて変更）
folder_path = "/content/drive/MyDrive/気象データと熱中症件数まとめ"  # 各都道府県のExcelファイルが入っているフォルダ

# 都道府県名リスト（47都道府県）
prefectures = [
    "北海道", "青森県", "岩手県", "宮城県", "秋田県", "山形県", "福島県",
    "茨城県", "栃木県", "群馬県", "埼玉県", "千葉県", "東京都", "神奈川県",
    "新潟県", "富山県", "石川県", "福井県", "山梨県", "長野県",
    "岐阜県", "静岡県", "愛知県", "三重県",
    "滋賀県", "京都府", "大阪府", "兵庫県", "奈良県", "和歌山県",
    "鳥取県", "島根県", "岡山県", "広島県", "山口県",
    "徳島県", "香川県", "愛媛県", "高知県",
    "福岡県", "佐賀県", "長崎県", "熊本県", "大分県", "宮崎県", "鹿児島県", "沖縄県"
]

# 結果を格納するリスト
results = []

# 各ファイルに対して処理
for pref in prefectures:
    filename = f"{pref}_気象データ.xlsx"
    file_path = os.path.join(folder_path, filename)

    try:
        # データ読み込み
        df = pd.read_excel(file_path)
        df["平均気温"] = pd.to_numeric(df["平均気温"], errors="coerce")
        df["平均湿度"] = pd.to_numeric(df["平均湿度"], errors="coerce")
        df["件数"] = pd.to_numeric(df["件数"], errors="coerce")
        df_clean = df.dropna(subset=["平均気温", "平均湿度", "件数"])

        if len(df_clean) < 3:
            print(f"{pref}: データ不足のためスキップ")
            continue

        # 回帰分析
        X = df_clean[["平均気温", "平均湿度"]]
        y = df_clean["件数"]
        model = LinearRegression()
        model.fit(X, y)
        y_pred = model.predict(X)

        # 結果の保存
        r2 = model.score(X, y)
        corr = np.corrcoef(y, y_pred)[0, 1]

        results.append({
            "都道府県": pref,
            "回帰係数_気温": model.coef_[0],
            "回帰係数_湿度": model.coef_[1],
            "切片": model.intercept_,
            "決定係数_R2": r2,
            "相関係数": corr
        })

    except Exception as e:
        print(f"{pref}: エラーが発生しました → {e}")

# 結果の出力
results_df = pd.DataFrame(results)
print(results_df)

# CSVファイルに保存（任意）
results_df.to_csv("都道府県別_重回帰分析結果.csv", index=False)


    都道府県    回帰係数_気温   回帰係数_湿度          切片   決定係数_R2      相関係数
0    北海道   3.313356 -0.131714  -48.650326  0.366074  0.605041
1    青森県   1.216261 -0.037108  -19.665152  0.386357  0.621577
2    岩手県   1.392076 -0.118730  -16.668310  0.473638  0.688213
3    宮城県   2.736194 -0.167437  -41.146909  0.500421  0.707404
4    秋田県   1.276255 -0.088841  -18.610034  0.449466  0.670422
5    山形県   1.251912 -0.076414  -19.216532  0.448019  0.669342
6    福島県   2.395279 -0.093863  -41.306414  0.511792  0.715396
7    茨城県   3.534626 -0.308635  -47.267941  0.504806  0.710497
8    栃木県   2.279429 -0.130281  -37.652941  0.500115  0.707188
9    群馬県   2.764569 -0.097231  -53.062351  0.521741  0.722317
10   埼玉県   8.678415 -0.543810 -131.233206  0.545871  0.738831
11   千葉県   6.503019 -0.009661 -132.783751  0.344061  0.586567
12   東京都  11.609893 -0.061013 -252.271828  0.458506  0.677131
13  神奈川県   7.014560 -0.280554 -131.747050  0.486538  0.697523
14   新潟県   2.931117 -0.210293  -45.580466  0.557387  0.746584
15   富山県

In [9]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression

# Google Drive のマウント先ディレクトリ
folder_path = "/content/drive/MyDrive/気象データと熱中症件数まとめ"  # ここに各都道府県のExcelファイルがあると仮定

# 出力先ファイルパス
output_path = "/content/drive/MyDrive/都道府県別_重回帰分析結果.csv"

# 都道府県名リスト
prefectures = [
    "北海道", "青森県", "岩手県", "宮城県", "秋田県", "山形県", "福島県",
    "茨城県", "栃木県", "群馬県", "埼玉県", "千葉県", "東京都", "神奈川県",
    "新潟県", "富山県", "石川県", "福井県", "山梨県", "長野県",
    "岐阜県", "静岡県", "愛知県", "三重県",
    "滋賀県", "京都府", "大阪府", "兵庫県", "奈良県", "和歌山県",
    "鳥取県", "島根県", "岡山県", "広島県", "山口県",
    "徳島県", "香川県", "愛媛県", "高知県",
    "福岡県", "佐賀県", "長崎県", "熊本県", "大分県", "宮崎県", "鹿児島県", "沖縄県"
]

results = []

for pref in prefectures:
    filename = f"{pref}_気象データ.xlsx"
    file_path = os.path.join(folder_path, filename)

    try:
        df = pd.read_excel(file_path)
        df["平均気温"] = pd.to_numeric(df["平均気温"], errors="coerce")
        df["平均湿度"] = pd.to_numeric(df["平均湿度"], errors="coerce")
        df["件数"] = pd.to_numeric(df["件数"], errors="coerce")
        df_clean = df.dropna(subset=["平均気温", "平均湿度", "件数"])

        if len(df_clean) < 3:
            print(f"{pref}: データ不足のためスキップ")
            continue

        X = df_clean[["平均気温", "平均湿度"]]
        y = df_clean["件数"]
        model = LinearRegression()
        model.fit(X, y)
        y_pred = model.predict(X)

        r2 = model.score(X, y)
        corr = np.corrcoef(y, y_pred)[0, 1]

        results.append({
            "都道府県": pref,
            "回帰係数_気温": model.coef_[0],
            "回帰係数_湿度": model.coef_[1],
            "切片": model.intercept_,
            "決定係数_R2": r2,
            "相関係数": corr
        })

    except Exception as e:
        print(f"{pref}: エラーが発生しました → {e}")

# 結果をDataFrameにして保存
results_df = pd.DataFrame(results)
results_df.to_csv(output_path, index=False, encoding="utf-8-sig")  # Excel互換の文字コードで保存

print("分析結果を保存しました:", output_path)


分析結果を保存しました: /content/drive/MyDrive/都道府県別_重回帰分析結果.csv
